In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re
import os

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [6]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [7]:
df['loc']

1     (9349 Collins Ave Apt 304, Surfside, FL 33154,...
2     (7454 Fisher Island Dr #7454, Miami Beach, FL ...
3     (2201 Collins Ave #1504, Miami Beach, FL 33139...
4     (16047 Collins Ave APT 2304, Sunny Isles Beach...
5     (101 20th St #1002, Miami Beach, FL 33139, USA...
6     (17475 Collins Ave UNIT 1002, Sunny Isles Beac...
7     (16051 Collins Ave APT 2602, Sunny Isles Beach...
8     (4000 Island Blvd #1, Aventura, FL 33160, USA,...
9     (3000 Island Blvd #3003, Aventura, FL 33160, U...
10    (901 Brickell Key Blvd APT 2406, Miami, FL 331...
Name: loc, dtype: object

In [8]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [9]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Chateau Ocean Condo\r9349 Collins Ave 304\rSur...,9/23/2021,9/6/2022,313,"$9,200,000.00",3232,"$2,846.53",Maria Oneide Gioia,Luxury Living Realty,Alina Meledina,AGK Realty LLC,Chateau Ocean Condo 9349 Collins Ave 304 Surfside,Chateau Ocean Condo,9349 Collins Ave 304 Surfside,"(9349 Collins Ave Apt 304, Surfside, FL 33154,...","(25.8830205, -80.1216893, 0.0)",25.883021,-80.121689,0.0
2,Oceanside at Fisher Island\r7454 Fisher Island...,2/21/2021,9/8/2022,541,"$5,400,000.00",2942,"$1,835.49",Evelyn Mackenzie,Aria Luxe Inc,Evelyn Mackenzie,Aria Luxe Inc,Oceanside at Fisher Island 7454 Fisher Island ...,Oceanside at Fisher Island,7454 Fisher Island Dr 7454 Miami Beach,"(7454 Fisher Island Dr #7454, Miami Beach, FL ...","(25.7617, -80.13568599999999, 0.0)",25.761700,-80.135686,0.0
3,W South Beach\r2201 Collins Ave 1504\rMiami Beach,8/2/2022,9/5/2022,17,"$3,950,000.00",2021,"$2,284.56",Christopher Adeleke,Douglas Elliman,Sophia Albrecht,Thine Agency,W South Beach 2201 Collins Ave 1504 Miami Beach,W South Beach,2201 Collins Ave 1504 Miami Beach,"(2201 Collins Ave #1504, Miami Beach, FL 33139...","(25.7975046, -80.1281362, 0.0)",25.797505,-80.128136,0.0
4,Turnberry Ocean Colony\r16047 Collins Ave 2304...,5/5/2022,9/6/2022,68,"$3,700,000.00",3735,$990.63,Steven Landau,Silmar Realty,Seth Feuer PA,"Compass Florida, LLC",Turnberry Ocean Colony 16047 Collins Ave 2304 ...,Turnberry Ocean Colony,16047 Collins Ave 2304 Sunny Isles Beach,"(16047 Collins Ave APT 2304, Sunny Isles Beach...","(25.9252659, -80.1216882, 0.0)",25.925266,-80.121688,0.0
5,Setai Resort & Residences\r101 20th St 1002\rM...,2/10/2022,9/9/2022,173,"$2,725,000.00",1213,"$2,246.50",Jeffrey Miller,One Sotheby's Int'l Realty,Reid Heidenry,One Sotheby's Int'l Realty,Setai Resort & Residences 101 20th St 1002 Mia...,Setai Resort & Residences,101 20th St 1002 Miami Beach,"(101 20th St #1002, Miami Beach, FL 33139, USA...","(25.7954398, -80.12805039999999, 0.0)",25.795440,-80.128050,0.0
6,Chateau Beach\r17475 Collins Ave 1002\rSunny I...,2/24/2021,9/9/2022,452,"$2,650,000.00",2164,"$1,224.58",Anna Reznik,GMT International Realty Inc,Anna Reznik,GMT International Realty Inc,Chateau Beach 17475 Collins Ave 1002 Sunny Isl...,Chateau Beach,17475 Collins Ave 1002 Sunny Isles Beach,"(17475 Collins Ave UNIT 1002, Sunny Isles Beac...","(25.9379684, -80.1207198, 0.0)",25.937968,-80.120720,0.0
7,Turnberry Ocean Colony\r16051 Collins Ave 2602...,1/27/2022,9/8/2022,180,"$2,190,000.00",2245,$975.50,Alejandro Mizrahi,P & I Florida LLC,Jacqueline Bedny,Beachfront Realty Inc,Turnberry Ocean Colony 16051 Collins Ave 2602 ...,Turnberry Ocean Colony,16051 Collins Ave 2602 Sunny Isles Beach,"(16051 Collins Ave APT 2602, Sunny Isles Beach...","(25.9258682, -80.1214311, 0.0)",25.925868,-80.121431,0.0
8,Williams Island\r4000 Island Blvd 1\rAventura,1/13/2022,9/7/2022,77,"$2,157,000.00",5136,$419.98,Edmund Bogen,Douglas Elliman,Jacqueline Cabral,Dezer Platinum Realty LLC,Williams Island 4000 Island Blvd 1 Aventura,Williams Island,4000 Island Blvd 1 Aventura,"(4000 Island Blvd #1, Aventura, FL 33160, USA,...","(25.9416056, -80.1348618, 0.0)",25.941606,-80.134862,0.0
9,Williams Island\r3000 Island Blvd 3003\rAventura,6/27/2022,9/6/2022,32,"$1,900,000.00",4407,$431.13,Nicole Shein,"Williams Island Realty, LLC",Tim Elmes,"Compass Florida, LLC",Williams Island 3000 Island Blvd 3003 Aventura,Williams Island,3000 Island Blvd 3003 Aventura,"(3000 Island Blvd #3003, Aventura, FL 33160, U...","(25.9406051, -80.1344671, 0.0)",25.940605,-80.134467,0.0
10,C0arbonel\r901 Brickell Key Blvd # 2406\rMiami,7/7/2022,9/7/2022,1,"$1,650,000.00",1944,$848.77,Damaso Medina,Capital Intl Realty LLC,Julio Oliveira,Elite International Realty Inc,C0arbonel 901 Brickell Key Blvd # 2406 Miami,C,0arbonel 901 Brickell Key Blvd # 240

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [14]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [10]:
df.at[10,'building_name']=('Carbonel')
df.at[10,'address_only']=('901 Brickell Key Blvd unit 2406')

In [11]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [12]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('September 4th - September 10th')

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [13]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [14]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [15]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_091222
